In [1]:
## This is demo for the inverse kinematics C++ version
## Author : Avadesh Meduri
## Date : 22/04/2021

import time
import numpy as np
from py_biconvex_mpc.ik.inverse_kinematics import InverseKinematics
from robot_properties_atlas.config import AtlasConfig
from py_biconvex_mpc.ik_utils.abstract_gait_generator import AbstractGaitGenerator

import pinocchio as pin



<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for boost::shared_ptr<hpp::fcl::CollisionGeometry> already registered; second conversion method ignored.


solo import 


In [2]:
robot = AtlasConfig.buildRobotWrapper()
rmodel = robot.model
rdata = robot.data
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [6]:
dt = 5e-2
T = 5

q0 = np.array([ 0.0, 0.0, 0.95, #base
                0.0, 0.0, 0.0, 1.0, #base quaternion
                0.0, #hip yaw
                0.0487, #hip forward/backward
                0.0, #hip tilt
                0.0, -1.2, 0.0, 0.0, 0.0, 0.0, 0.0, #left arm
                0.0, 1.2, 0.0, 0.0, 0.0, 0.0, 0.0, #right arm
                0.0, 0.0, 0.0, #left hip abductors
                0.0, 0.0, 0.0, #left knee, ankle tilt fwd, ankle tilt side
                0.0, 0.0, 0.0, #right hip abductors
                0.0, 0.0, 0.0]) #right knee, right ankle tilt fwd, right ankle tilt side;
v = np.zeros(rmodel.nv)
x0 = np.concatenate([q0, np.zeros(rmodel.nv)])

stateWeights = np.array([0.] * 3 + [50.] * 3 + [0.01] * (rmodel.nv - 6) \
                        + [10.] * 6 + [1.0] *(rmodel.nv - 6))

# print(robot.model.nq, robot.model.nv)

des_pos_left = np.tile(np.array([0.0,   0.1115,  0]), (int(T/dt),1))
des_pos_right = np.tile(np.array([0.0,   -0.1115,  0]), (int(T/dt),1))

des_vel_left = np.tile(np.array([0.,   0,  0]), (int(T/dt),1))
des_vel_right = np.tile(np.array([0.,   -0,  0]), (int(T/dt),1))

des_com_pos = np.tile(np.array([0.,   0,  1.2]), (int(T/dt),1))
des_com_pos[:,0] = 0.1*np.linspace(0, len(des_com_pos), len(des_com_pos))

des_mom = np.tile(np.array([0.,   0,  0.0, 0, 0, 0]), (int(T/dt),1))
des_mom[:,0] = 0.1

sl = np.array([0.5, 0, 0])
st = 0.5

gg = AbstractGaitGenerator(rmodel, rdata, AtlasConfig.urdf_path, T, dt)
gg.create_swing_foot_task(des_pos_left[0], des_pos_left[0] + sl, 0, st, 0.1, "l_foot", "L_step", 1e3)
gg.create_contact_task(des_pos_right[0], 0, st, "r_foot", "R_step", 1e5)
gg.create_contact_task(des_pos_left[0] + sl, st, T, "l_foot", "L_step", 1e5)
gg.create_swing_foot_task(des_pos_right[0], des_pos_right[0] + 2*sl, st, 2*st, 0.1, "r_foot", "R_step", 1e3)
gg.create_contact_task(des_pos_right[0] + 2*sl, 2*st , T, "r_foot", "R_step", 1e5)

gg.create_centroidal_task(des_mom, 0, T, "mom track", 1e1)

xs, us = gg.optimize(x0, stateWeights, x0, wt_xreg=5e-4, wt_ureg=1e-5)
# np.savez("../motion_planner/dat_file/ik", xs = xs)


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. inverse_kinematics_cpp.InverseKinematics(arg0: str, arg1: int)

Invoked with: '/home/ameduri/.local/lib/python3.8/site-packages/robot_properties_atlas/robot_properties_atlas/urdf/atlas.urdf', 0.05, 5

In [5]:
for i in range(len(xs)):
    time.sleep(0.06)
    viz.display(xs[i][:rmodel.nq])

NameError: name 'xs' is not defined

In [9]:
rmodel

Nb joints = 31 (nq=36,nv=35)
  Joint 0 universe: parent=0
  Joint 1 root_joint: parent=0
  Joint 2 back_bkz: parent=1
  Joint 3 back_bky: parent=2
  Joint 4 back_bkx: parent=3
  Joint 5 l_arm_shz: parent=4
  Joint 6 l_arm_shx: parent=5
  Joint 7 l_arm_ely: parent=6
  Joint 8 l_arm_elx: parent=7
  Joint 9 l_arm_wry: parent=8
  Joint 10 l_arm_wrx: parent=9
  Joint 11 l_arm_wry2: parent=10
  Joint 12 r_arm_shz: parent=4
  Joint 13 r_arm_shx: parent=12
  Joint 14 r_arm_ely: parent=13
  Joint 15 r_arm_elx: parent=14
  Joint 16 r_arm_wry: parent=15
  Joint 17 r_arm_wrx: parent=16
  Joint 18 r_arm_wry2: parent=17
  Joint 19 l_leg_hpz: parent=1
  Joint 20 l_leg_hpx: parent=19
  Joint 21 l_leg_hpy: parent=20
  Joint 22 l_leg_kny: parent=21
  Joint 23 l_leg_aky: parent=22
  Joint 24 l_leg_akx: parent=23
  Joint 25 r_leg_hpz: parent=1
  Joint 26 r_leg_hpx: parent=25
  Joint 27 r_leg_hpy: parent=26
  Joint 28 r_leg_kny: parent=27
  Joint 29 r_leg_aky: parent=28
  Joint 30 r_leg_akx: parent=29